# 普通名詞の獲得
本ノートブックでは普通名詞の獲得について仮説を立てて検証する。

最初に検証する仮説はこちら:

* 辞書ベースでラベリングした際、正解部分以外にラベリングした場合は普通名詞である可能性が高い


まずは必要なモジュールをインポートする。

In [15]:
import os
import sys
sys.path.append('..')

from seqeval.metrics import f1_score

from entitypedia.evaluation.converter import to_iob2
from entitypedia.knowledge_ner.recognizer import KnowledgeBaseRecognizer
from entitypedia.corpora.wikipedia.extractor import load_jsonl

まずは辞書ベースの分類器を読み込む。

In [16]:
recognizer = KnowledgeBaseRecognizer()
file_path = '../data/interim/title_entity.jsonl'
for d in load_jsonl(file_path):
    entity, label = list(d.items())[0]
    sub_type = label.split('/')[-1]
    label = label.split('/')[0]
    recognizer.add_entity(entity, label)
    recognizer.add_word(entity, sub_type, page_url='', image_url='')
recognizer.build()

除外するタイプを定義する。

In [17]:
remove_types = {'volume', 'period_date', 'percent', 'url', 'service', 'multiplication',
                'n_person', 'school_age', 'seismic_intensity', 'period_month',
                'phone_number', 'rank', 'n_animal', 'countx_other', 'point',
                'periodx_other', 'calorie', 'space', 'period_time', 'n_country',
                'n_product', 'numex_other', 'latitude_longtitude', 'id_number',
                'n_flora', 'facility_part', 'temperature', 'weight', 'age', 'water_root',
                'n_natural_object_other', 'intensity', 'time', 'n_facility',
                'n_organization', 'postal_address', 'period_year', 'ordinal_number',
                'physical_extent', 'speed', 'measurement_other', 'seismic_magnitude',
                'n_event', 'period_week', 'frequency', 'ignored', 'stock', 'n_location_other'}

次に正解データを読み込む。

In [18]:
BASE_DIR = '../data/raw/corpora'
mainichi_dir = os.path.join(BASE_DIR, 'mainichi')
X_true, y_true = to_iob2(mainichi_dir, remove_types)

正解データを読み込んだら、辞書ベース分類器を用いて、ラベル付する。

In [19]:
y_pred = [recognizer.iob2(text) for text in X_true]

以上で仮説検証の準備ができた。ここからは実際に検証していく。

まずは正解ラベルが付いていない箇所にラベル付している部分を洗い出す。

In [20]:
for y_p, y_t, char in zip(y_pred[0], y_true[0], X_true[0]):
    if y_t == 'O' and y_p != 'O':
        print('{}\t{}'.format(char, y_p))

国	B-other
際	I-other
な	B-weapon
ど	B-language_other
の	B-government
基	B-law
本	I-law
す	B-weapon
は	B-weapon
の	B-government
会	I-city
見	I-city
で	B-language_other
と	B-book
す	B-weapon
る	B-city
所	B-city
を	B-position_vocation
発	B-other
表	I-other
し	B-weapon
た	B-weapon
月	B-planet
に	B-weapon
を	B-position_vocation
む	B-city
グ	B-language_other
ル	I-language_other
ー	I-language_other
が	B-language_other
発	B-other
表	I-other
す	B-weapon
る	B-city
の	B-government
基	B-law
本	I-law
的	B-drug
な	B-weapon
え	B-weapon
を	B-position_vocation
し	B-weapon
た	B-weapon
も	B-city
の	B-government
わ	B-city
が	B-language_other
国	B-country
に	B-weapon
ふ	B-weapon
さ	B-weapon
わ	B-city
し	B-weapon
い	B-weapon
国	B-other
際	I-other
に	B-weapon
よ	B-position_vocation
る	B-city
世	B-continental_region
界	I-continental_region
平	B-war
和	I-war
の	B-government
創	B-position_vocation
造	I-position_vocation
と	B-book
っ	B-sport
た	B-weapon
軍	B-military
事	I-military
分	B-other
野	I-other
の	B-government
国	B-other
際	I-other
な	B-weapon
ど	B-language_other
つ	B-position_voc

上記を分析すると以下のことがわかる。

* 曖昧さ回避ページが多い
* 全体的には普通名詞であることが多い
* 固有名詞の場合もある(e.g. 新 -> 昔の中国の国名)

仮説としては悪くなさそうなので、さらに検証を進めていく。
上記は文字単位でついているので、単語単位に変換して分析する。

In [21]:
from seqeval.metrics.sequence_labeling import get_entities

chars = []
tags = []
for pred, true, doc in zip(y_pred, y_true, X_true):
    for p, t, char in zip(pred, true, doc):
        if t == 'O' and p != 'O':
            chars.append(char)
            tags.append(p)
            
entities = get_entities(tags)

In [22]:
words = []
for _, begin_offset, end_offset in entities:
    end_offset += 1
    word = ''.join(chars[begin_offset: end_offset])
    words.append(word)
    
words[0:10]

['国際', 'な', 'ど', 'の', '基本', 'す', 'は', 'の', '会見', 'で']

ここまでで、ラベル付結果を単語単位に変換しました。

得られた単語の統計情報を確認してみます。確認する統計情報は以下の通り。

* トークン数(延べ単語数)
* タイプ数(異なり語数)
* 各単語の頻度

In [23]:
from collections import Counter

freq = Counter(words)
freq100 = sum(v for _, v in freq.most_common(n=100))

print('Num of tokens: {}'.format(len(words)))
print('Num of types  : {}'.format(len(set(words))))
print('{} / {}'.format(freq100, len(words)))

for word, fr in freq.most_common():
    print('{}\t{}'.format(word, fr))

Num of tokens: 1598859
Num of types  : 17127
1277500 / 1598859
の	115878
に	67170
た	61589
は	58124
を	57959
い	57238
が	53095
と	51385
る	51022
し	49934
で	48622
て	44172
な	40300
か	29435
っ	26939
れ	25324
ら	24059
も	21767
す	20172
こ	17203
う	16827
り	16577
ま	16507
だ	16211
き	13218
く	13128
さ	12554
め	11714
け	10907
ど	10833
あ	10480
ん	9288
え	8512
そ	8001
つ	7873
や	7207
わ	6987
年	6726
せ	5891
分	5402
ち	4739
み	4529
ば	4294
日	4284
ろ	4186
的	3988
よう	3820
よ	3732
お	3587
じ	3274
本	3266
ず	3219
合	3210
間	2974
べ	2941
内	2907
へ	2891
対	2781
場	2757
目	2659
方	2607
げ	2500
要	2395
ほ	2296
新	2268
国	2267
定	2189
手	2167
取	2095
強	2089
受	2052
連	1974
地	1968
とも	1955
月	1942
体	1941
金	1915
び	1913
調	1828
む	1752
午	1749
半	1693
数	1657
現在	1639
明	1602
カ	1595
事	1567
なかった	1550
化	1505
問題	1447
ご	1392
ぶ	1345
ね	1334
交渉	1325
向	1319
政府	1302
私	1293
業	1273
気	1270
所	1262
反	1241
文	1204
より	1202
ぐ	1116
心	1101
何	1101
ぎ	1091
まれ	1074
市	1064
田	1059
今年	1055
町	1049
指	1039
無	1024
山	1016
女性	1001
死	993
結	989
引	971
時間	967
著作権	952
理	919
両	902
いない	892
りの	876
外	869
世界	862
企業	838


和歌山市	27
須磨区	27
市街地	27
痴漢	27
情報公開	26
議長	26
日付	26
経由	26
しつけ	26
去年	26
敗戦	26
デリバティブ	26
教授	26
国籍	26
持続	26
出馬表	26
参拝	26
飛行機	26
繁殖	26
言語	26
疎開	26
漁	26
汗	26
現行犯	26
大部	26
視聴者	26
暫	26
日朝	26
亡命	26
洗い	26
医薬品	26
半導体	26
まじめ	26
銃	26
布団	26
都心	26
偶然	26
捕虜	26
週末	26
ソプラノ	26
二十九歳	26
累	26
一ツ橋	26
トンネル	26
公団	26
投手	26
ありがとう	26
スカ	26
ゲス	26
ペット	26
訴状	26
のん	26
現	26
再開発	26
盗塁	26
代打	26
飛躍	25
高齢化社会	25
党大会	25
市民団体	25
回転	25
派閥	25
区域	25
幸福	25
おじ	25
ハイテク	25
奨学金	25
森林	25
摩擦	25
芽	25
動力	25
時計	25
少数民族	25
滑走路	25
クイズ	25
本質	25
解釈	25
ランキング	25
一行	25
国交	25
放射能	25
友だち	25
栄養	25
ツケ	25
規則	25
ぺ	25
神宮	25
本土	25
絶望	25
俳句	25
控訴	25
住宅ローン	25
郵便局	25
勝ち点	25
行部	25
住宅金融専門会社	25
満塁	25
毒ガス	25
党派	24
塔	24
水面	24
ちから	24
バックアップ	24
立法	24
投機	24
照明	24
チャリティー	24
道場	24
教会	24
リーグ戦	24
合宿	24
奈良	24
知名度	24
広報	24
探検	24
翻訳	24
襲撃	24
負債	24
闇	24
着陸	24
後援会	24
朝食	24
定義	24
大和	24
埼玉県	24
鑑定	24
愛好	24
演奏会	24
支持率	24
ヤード	24
局長	24
脚本	24
ベンチ	24
灰	24
昌	24
審議会	24
プレーオフ	24
オリジナル	24
国産	24
仕入	24
冠	24
打点	24
鬼	24
ユーザー	24
豊中市	24
陸軍	24
借家	24
強度	24
完投	24
社説	23
新制	23
民党	23
世代交代	23
見出し	23

巻き返し	8
保安	8
大中	8
前哨戦	8
確率	8
取組	8
ほのぼの	8
コイ	8
ドラフト会議	8
電池	8
留置	8
コーデ	8
水道管	8
放浪	8
土地神話	8
タレント	8
売れ筋	8
ライター	8
間取り	8
ガル	8
意志	8
閉会	8
ギャン	8
還暦	8
ナラ	8
共同体	8
突き出し	8
定性	8
地名	8
個体	8
いただきます	8
草刈り	8
駆除	8
宇治市	8
地殻変動	8
音楽祭	8
らする	8
アナログ	8
タネ	8
メガ	8
十両	8
引き落とし	8
福岡県	8
石炭	8
観念	8
飛び降り	8
違憲	8
語録	8
スマ	8
舗装	8
どん底	8
ポル	8
立地	8
エレクトロ	8
ンビニ	8
ストライキ	8
トロ	8
フォークボール	8
理科	8
肖像	8
彦根市	8
賓	8
寝室	8
執着	8
ガス室	8
演習	8
石垣	8
クッション	8
欠陥	8
たち	8
キチン	8
淀	8
電線	8
総意	8
倉敷市	8
伏見区	8
北川	8
東成区	8
羽曳野市	8
地球	8
岡山市	8
城東区	8
西成区	8
下山手通	8
東川崎町	8
ハードル	8
進駐	8
嘘	8
自分史	8
歩行	8
野焼き	8
放射性物質	8
協調介入	8
打数	8
国家権力	8
シンポ	8
生き埋め	8
消毒	8
ボン	8
突然死	8
仏教	8
煙突	8
開発途上国	8
刀	8
化学物質	8
文学賞	8
洗浄	8
用途	8
堰	8
降板	8
オーバル	8
社団法人	8
追っかけ	8
紀伊国	8
工作機械	8
主砲	8
サリン	8
オルガン	8
追加公認	8
シャンソン	8
両替	8
靴下	8
化石	8
水死	8
時制	8
クロージャ	8
旅費	8
タイガ	8
三党合意	8
中元	8
地形	8
他界	8
掘削	8
環境基準	8
表現の自由	8
鉱床	8
自由形	8
能面	8
傘立て	8
イドラ	8
藤本定義	8
標準化	7
年男	7
天国	7
うなり	7
直接選挙	7
政務	7
年越し	7
苗木	7
商品開発	7
歌唱	7
ショッピングセンター	7
テナント	7
カウントダウン	7
五歩	7
アブ	7
新聞紙	7
六番	7
説教	7
スペイン	7
スケート	7
窒息	7
疾走	7
日焼け	7
医師	7
聖地	7
征服	7
初夢	7
四輪駆動	7
パニック

義理チョコ	3
部員	3
斑鳩町	3
バレン	3
帽子	3
協同組合	3
米子市	3
庄町	3
大和郡山市	3
ライオンズクラブ	3
情報システム	3
歩行者	3
重奏	3
鳴門市	3
無縁仏	3
呉服	3
垣見隆	3
山口鶴男	3
久保亘	3
文子	3
扇動	3
ベアゼロ	3
高不況	3
わが家で	3
アニメーション	3
日部	3
花粉	3
酪農	3
ストリ	3
日興	3
見本市	3
金融調節	3
尻字	3
油田	3
洙	3
新栄	3
岡崎市	3
錦町	3
大坪	3
丸之内	3
紺屋	3
泉佐野市	3
札幌市	3
田谷	3
つくば市	3
阿見町	3
鋳物	3
坂戸市	3
美浜区	3
金沢区	3
都筑区	3
貝塚	3
土橋	3
神田錦町	3
上大崎	3
立川市	3
国分寺市	3
本荘	3
北浜	3
博多駅	3
ろうそく	3
弁者	3
ペルー	3
法の下の平等	3
法律関係	3
ボンベ	3
偽造旅券	3
好きだ。	3
予科	3
長屋	3
スキンシップ	3
中南	3
ホットライン	3
や団	3
殿堂	3
四谷怪談	3
スコープ	3
粘度	3
カー用品	3
ハイフン	3
ディーゼルエンジン	3
ローマ	3
ジア	3
トガ	3
真空	3
マンダ	3
現代詩	3
プレイス	3
アリエ	3
ラヤ	3
博展	3
紀尾井町	3
発光	3
風向	3
荘厳	3
ベッドタウン	3
清浄	3
充電器	3
縮瞳	3
交通安全	3
突然変異	3
司法研修所	3
法務省	3
郷土料理	3
三府	3
河口堰	3
伸子	3
占い	3
憎しみ	3
パラボラアンテナ	3
缶コーヒー	3
書式	3
ブラシ	3
尊厳死	3
地域医療	3
ささえ	3
ピッチング	3
満塁本塁打	3
流し打ち	3
出塁	3
虚実	3
市電	3
化学式	3
わいら	3
パントマイム	3
ボウリング	3
ヤシ	3
香料	3
東国	3
サンドイッチ	3
シンボルマーク	3
聖人	3
戯画	3
シアターアプル	3
アゴラ	3
ヴィオラ	3
リックス	3
精米	3
平仮名	3
カルフ	3
公共機関	3
連続テレビ小説	3
二次災害	3
黒人	3
公序良俗	3
ガール	3
理数	3
しより	3
使い捨て	3
分別収集	3
あらき	3
乾和	3
情報機関	3
至上命題	3
燃料棒	3
胴体	3
政庁	3
突撃	3
破壊活動	3
保釈	3
川

暗算	2
ズサ	2
課徴金	2
親分	2
防犯	2
空手道	2
機会均等	2
乗車券	2
自由選挙	2
ミニ四駆	2
抄	2
留美	2
われら	2
商工会	2
バリトン	2
求婚	2
胸毛	2
岡本文弥	2
杉村春子	2
加藤剛	2
ビルマの竪琴	2
敬虔	2
弦楽合奏	2
無我	2
肉食	2
辻堂神台	2
聖堂	2
長崎の鐘	2
キログラム	2
戦争犯罪	2
たいまつ	2
妙法	2
交通整理	2
野田健	2
林則清	2
柳井俊二	2
外務審議官	2
自由社	2
農地改革	2
絵画教室	2
分目	2
環境学	2
南品川	2
江坂	2
競泳	2
背泳ぎ	2
平泳ぎ	2
珠子	2
アフラ	2
ホームページ	2
門外	2
警報システム	2
地盤調査	2
金的	2
佐藤美枝子	2
忌避	2
ツルハ	2
懲戒処分	2
全身麻酔	2
発泡スチロール	2
脳死	2
地域密着	2
劣後ローン	2
コク	2
さんふらわあ	2
慢性疾患	2
共済	2
神戸銀行	2
司法書士	2
宅地建物取引業	2
つづら折り	2
雑木	2
羽曳が丘	2
テーマパーク	2
ゴルファー	2
漬け	2
リボン	2
シール	2
エストニア	2
デザート	2
マリネ	2
羽毛	2
アクション映画	2
先崎	2
スキャット	2
スキャットマン・ジョン	2
オリコン	2
小法廷	2
調査船	2
自己資本比率	2
木場	2
聴覚	2
炎天	2
ブラウン管	2
再軍備	2
軍事同盟	2
か抑止	2
共通外交・安全保障政策	2
欧州連合	2
ステレオ	2
総需要	2
相互銀行	2
ペンギン	2
新潮社	2
遠近	2
寝具	2
出	2
アウトボクシング	2
個人メドレー	2
メドレーリレー	2
別大	2
製油	2
オークマ	2
オーム	2
ドーム山	2
無線局	2
一個人	2
前庭	2
バリアフリー	2
学齢	2
フリースクール	2
発電機	2
新品	2
虚飾	2
オレたち	2
迷宮	2
非核地帯	2
外交交渉	2
神楽坂	2
ミール	2
チベット	2
平幹二朗	2
ジョビジョバ	2
浅草	2
加藤健一	2
スカラ座	2
北とぴあ	2
二部作	2
銃殺	2
法解釈	2
分一	2
シュプレヒコール	2
植木鉢	2
ト	2
核兵器	2
ラグーン	2
坂本堤	2
未収金	2
合性論	2
分子構造	2
黒板	2
政治哲学	2
マー

ミテ	1
芦屋市民センター	1
ベガ	1
兵庫県立文化体育館	1
東条町	1
明石市立天文科学館	1
摩耶山町	1
若竹	1
灘北通	1
大映	1
外国人学校	1
指圧	1
の女性	1
夜食	1
人工呼吸器	1
電熱	1
はめ込み	1
ガーデンシティ	1
モニタリング	1
情報化社会	1
外資規制	1
対外債務	1
夢のなか	1
つないだ手	1
かくれんぼ	1
ターパン	1
軍事技術	1
三すくみ	1
衰弱死	1
のように	1
海上封鎖	1
ハエ	1
技術と	1
比肩	1
正殿	1
神社建築	1
掘立柱	1
五部作	1
オールカラー	1
全一	1
代将	1
解釈学	1
自己実現	1
濡れ	1
瘴気	1
表と裏	1
基準改定	1
富士吉田市	1
新倉	1
不忍池	1
地久	1
富士見市	1
関沢	1
地震予知連絡会	1
帯状分布	1
国立病院	1
消臭剤	1
名寄市	1
バーチャルリアリティ	1
ストーブ	1
豊田通商	1
アクロ	1
植田辰哉	1
宮崎謙彦	1
江越由佳	1
り代表	1
記録会	1
の地域	1
高橋千恵美	1
日本ケミコン	1
木村泰子	1
岩田屋	1
弘山晴美	1
エチオピア	1
メンディ	1
ユリア	1
ダニ	1
上野孝	1
十条	1
ハシェ	1
ピストル	1
射撃競技	1
木田知宏	1
斉藤浩哉	1
ロサリオ	1
サウスポー	1
アーリントンカップ	1
エイシンバーリン	1
ダイタク	1
中山牝馬ステークス	1
ホクトベガ	1
金奉洙	1
アミー	1
額縁	1
北の丸公園	1
ラメ	1
フィーリ	1
群馬県	1
北群馬郡	1
伊香保町	1
塩漬け	1
圏外	1
インスタント食品	1
スノーモービル	1
冥福	1
予備役	1
地熱	1
護衛空母	1
岩園町	1
抗争事件	1
銃社会	1
センバ	1
高知県	1
宿毛市	1
焼却炉	1
フィリピン航空	1
見	1
ブラックマンデー	1
内助の功	1
横浜アリーナ	1
代々木	1
長崎県立総合体育館	1
帯広の森アイスアリーナ	1
谷川岳	1
秩父宮	1
友綱	1
押尾川	1
春日野	1
剣山	1
間垣	1
応和	1
輸入割当制	1
法の支配	1
教育大学	1
干物	1
性教育	1
野口健	1
亜細亜大学	1
国際関係学部	1
立教英国学院	1
ヨーロッパ大陸	1
モンブラン	1
一貫教育	1
物性	

渡辺毅	1
野々村芳和	1
土橋正樹	1
大石尚哉	1
布部陽功	1
安藤正裕	1
名波浩	1
大岩剛	1
内田利広	1
深川友貴	1
萩野英明	1
大木勉	1
茶野隆行	1
吉田孝行	1
久保山由清	1
中田英寿	1
加藤泰明	1
山根巌	1
足技	1
終戦の	1
パタ	1
ナポ	1
カラト	1
シット	1
タウィー	1
フライ級	1
山口圭司	1
西條	1
岳人	1
琉球銀行	1
打撃	1
寝違え	1
チャンスメーカー	1
メロディア	1
ワードローブ	1
懐中時計	1
不信任決議	1
恒星	1
浮桟橋	1
手押しポンプ	1
オーケー	1
転落死	1
事故調査	1
の尾根	1
マントラ	1
電気街	1
国籍差別	1
重要無形民俗文化財	1
稚児	1
幕の内弁当	1
月経	1
履物	1
就職協定	1
首都圏	1
国土軸	1
支援団体	1
製氷	1
海上保険	1
連音	1
ザ・フェニックスホール	1
島えりか	1
美奈子	1
ビアマグ	1
電子手帳	1
ウレタン	1
緩衝材	1
準拠	1
の治療	1
脂肪組織	1
水晶体	1
網膜	1
分注	1
枡	1
湯浅町	1
印章	1
踊れ	1
神の子	1
金光教	1
力王	1
ボクシングジム	1
安達哲	1
指定金融機関	1
蓮池町	1
神戸高速鉄道	1
須磨浦公園	1
内部告発	1
四苦八苦	1
ミニ政党	1
の政党	1
共同企業体	1
診療科	1
ピアノソナタ	1
コリオラン	1
ピンズ	1
麻雀	1
晴眼者	1
高橋展子	1
佐藤ギン子	1
新木栄吉	1
西山勉	1
合資会社	1
古垣鉄郎	1
大隈信幸	1
パナマ	1
津田正夫	1
日本新聞協会	1
クウェート	1
日本原子力発電	1
波多野裕造	1
カタール	1
編集委員	1
軍港	1
弾道ミサイル	1
対戦勝記念日	1
トカット	1
人称	1
三人称	1
近親相姦	1
意地悪	1
優越感	1
ハンガリー	1
ヌシ	1
悪童日記	1
環境デザイン	1
ランドスケープ	1
美術	1
モダニズム	1
天窓	1
社会史	1
廃坑	1
新しい地図	1
ゆうゆ	1
辻占	1
ビーズ	1
行人	1
薪能	1
スイートルーム	1
公設市場	1
豚肉	1
城門	1
デンカ	1
ビーナスライン	1
ウナギ	1
歴史通	1
武士道	1
ヒロイック	1
キルト	1
現代狂言	1
使用者責

勅使河原宏	1
岸田今日子	1
加山雄三	1
白い巨塔	1
田宮二郎	1
東野英治郎	1
拝領妻始末	1
司葉子	1
神々の深き欲望	1
嵐寛寿郎	1
沖山秀子	1
心中天網島	1
右衛門	1
岩下志麻	1
井川比佐志	1
倍賞千恵子	1
忍ぶ川	1
栗原小巻	1
津軽じょんがら節	1
斎藤耕一	1
江波杏子	1
織田あきら	1
加藤嘉	1
佐分利信	1
小川真由	1
不毛地帯	1
幸福の黄色いハンカチ	1
高倉健	1
あゝ野麦峠	1
原田美枝子	1
泥の河	1
小栗康平	1
風間杜夫	1
戦場のメリークリスマス	1
大島渚	1
坂本龍一	1
薬師丸ひろ子	1
三田佳子	1
海と毒薬	1
奥田瑛二	1
渡辺謙	1
伊丹十三	1
宮本信子	1
となりのトトロ	1
宮崎駿	1
黒い雨	1
田中好子	1
永瀬正敏	1
周防正行	1
本木雅弘	1
月はどっちに出ている	1
崔洋一	1
岸谷五朗	1
ルビー・モレノ	1
全身小説家	1
原一男	1
日本ソフトボールリーグ	1
西日本シロアリ	1
船橋市総合体育館	1
守備率	1
ゴールデングラブ賞	1
斎藤雅樹	1
工藤公康	1
古田敦也	1
捕手	1
駒田徳広	1
一塁手	1
立浪和義	1
二塁手	1
小久保裕紀	1
石井琢朗	1
三塁手	1
馬場敏史	1
野村謙二郎	1
田中幸	1
外野手	1
緒方孝市	1
音重鎮	1
田口壮	1
尾崎将司	1
田中秀道	1
友利勝良	1
太平洋クラブ	1
ダンロップ	1
よみうり	1
ゲンス	1
八代将軍吉宗	1
水戸黄門	1
まだ恋は始まらない	1
人生は上々だ	1
筋肉	1
スーパー競馬	1
菊花賞	1
ふたりのビッグショー	1
ミュージックステーション	1
ミュージックフェア	1
週日	1
路線図	1
難燃剤	1
芸術大学	1
常盤平	1
篠崎町	1
公職追放	1
不動産登記	1
カッターナイフ	1
冬将軍	1
夜泣き	1
投資顧問会社	1
熊手	1
ファンタジー	1
文芸雑誌	1
診療報酬	1
選挙違反	1
風俗店	1
アメリカン・ドリーム	1
幕張メッセ	1
東京モーターショー	1
木津信用組合	1
忘れないよ	1
オールディーズ	1
レゲエ	1
サントン	1
紙吹雪	1
ガタピシ	1
ワーウ	1
ワード	1
リモコン	1
ビッグスリー	1
キープレフト	1
眩	1
フロントグ

全体的な感想:

* 曖昧さ回避ページが多い
* 固有名詞も結構多いように感じる

固有名詞が多いのはどういうことなんだろう？

* 正解データの質が悪い?
  * 辞書ベースではついたのに正解にはついていない
* 正解データの読み込み方にバグがある?
* `to_iob2`関数でignoredを弾いたのに問題がある?

おそらく、`to_iob2`関数に原因がある。というわけで修正して再びやってみる。

In [24]:
X_true, y_true = to_iob2(mainichi_dir, {})
y_pred = [recognizer.iob2(text) for text in X_true]

chars = []
tags = []
for pred, true, doc in zip(y_pred, y_true, X_true):
    for p, t, char in zip(pred, true, doc):
        if t == 'O' and p != 'O':
            chars.append(char)
            tags.append(p)
            
entities = get_entities(tags)

words = []
for _, begin_offset, end_offset in entities:
    end_offset += 1
    word = ''.join(chars[begin_offset: end_offset])
    words.append(word)
    
freq = Counter(words)
freq100 = sum(v for _, v in freq.most_common(n=100))

print('Num of tokens: {}'.format(len(words)))
print('Num of types  : {}'.format(len(set(words))))
print('{} / {}'.format(freq100, len(words)))

for word, fr in freq.most_common():
    print('{}\t{}'.format(word, fr))

Num of tokens: 1476862
Num of types  : 11106
1216028 / 1476862
の	110698
に	65694
た	60210
を	56702
は	55999
い	55505
が	51986
と	50167
る	50050
し	48526
で	46487
て	42777
な	39332
か	28586
っ	26168
れ	24779
ら	23372
も	21173
す	19395
こ	16818
う	16382
だ	15930
り	15928
ま	14765
く	12685
き	11810
め	11421
さ	11216
け	10547
ど	10486
あ	10242
え	8312
そ	7815
ん	7234
つ	7077
や	7029
わ	6771
せ	4756
ち	4555
み	4296
ば	4134
ろ	4053
的	3923
よう	3706
よ	3606
分	3280
年	3186
お	3165
じ	3138
ず	3074
合	3049
べ	2883
へ	2791
対	2661
内	2650
場	2530
方	2464
げ	2392
要	2352
ほ	2172
定	2110
取	2034
強	2006
受	1979
手	1956
とも	1928
間	1928
日	1909
国	1901
新	1897
目	1875
体	1831
地	1822
び	1821
む	1676
金	1665
連	1632
調	1632
本	1534
なかった	1534
事	1491
化	1485
明	1426
問題	1424
ぶ	1308
ご	1297
ね	1295
政府	1288
向	1259
私	1245
気	1215
業	1214
反	1155
より	1124
数	1066
ぐ	1059
今年	1034
ぎ	1034
まれ	1025
何	1024
心	990
指	976
結	967
所	960
女性	943
引	939
いない	879
市	860
無	856
死	855
理	854
りの	842
両	841
月	840
発表	813
企業	813
世界	811
止	797
論	776
事件	766
任	746
有	746
額	738
担	735
はい	730
ひ	712
運	705
一方	696
参加	686
可能	674
現在	

イカ	16
重し	16
風呂	16
復調	16
鑑賞	16
カップル	16
同州	16
虐殺	16
加害者	16
反動	16
明暗	16
名刺	16
予言	16
スリ	16
プライバシー	16
副作用	16
同情	16
終止符	16
引用	16
晩年	16
エッセ	16
成金	16
飢え	16
計測	16
シングル	16
綱領	16
我が家	16
おしゃれ	16
区分	16
事業者	16
信じる	16
共犯	16
患	16
マフィア	16
紙芝居	16
文化人	16
素人	16
株主	16
善意	16
奴	16
抗告	16
爆破	16
ライフライン	16
領土	16
スカ	16
続投	16
医院	16
信託	16
暴走族	16
フレーズ	15
技術開発	15
傍聴	15
還元	15
次元	15
社会問題	15
除外	15
未来を	15
だんだん	15
奇妙	15
乳	15
奪取	15
土台	15
気をつけ	15
ケール	15
少年時代	15
食堂	15
集合	15
勇	15
承諾	15
版画	15
タブー	15
戦車	15
新年	15
パチンコ	15
原理	15
液体	15
キリ	15
凡	15
威力	15
偏	15
データベース	15
端末	15
くじ	15
花火	15
開店	15
屋台	15
生物	15
丈	15
第三セクター	15
スローガン	15
診察	15
虫	15
ベストセラー	15
登記	15
昨日	15
不登校	15
未知	15
筒	15
妊娠	15
文化財	15
水平	15
扉	15
水深	15
女房	15
定価	15
航空会社	15
ドーム	15
駐車	15
官邸	15
マージ	15
包み	15
この女	15
無線	15
スクリーン	15
停滞	15
句集	15
外野	15
会館	15
設備工事	15
ミシン	15
興行	15
墨	15
機動	15
郡	15
株主総会	15
水害	15
人として	15
児童相談所	15
地震波	15
証人喚問	15
スプレー	15
三塁打	15
紙飛行機	15
げす	14
召集	14
生活者	14
根源	14
軍事力	14
貨物船	14
傑作	14
共働き	14
マッチ	14
名門	14
太鼓	14
宅配便	14
郵便物	14
氷	14
砂漠	14
何者	14
学校教育	14
ガキ大将	14
単身赴任	14
経歴

音質	4
リニューアル	4
ゆるい	4
オムニバス	4
釣り糸	4
風船	4
国際結婚	4
商都	4
女たち	4
レーダー	4
地方政府	4
エキス	4
出先機関	4
高レベル放射性廃棄物	4
動脈	4
ノビ	4
宮殿	4
防空	4
ファンダメンタル	4
ガイドブック	4
熱量	4
鐘を鳴らして	4
イブニング	4
査証	4
再審	4
テコ入れ	4
夜景	4
時速	4
量化	4
献立	4
レコードプレーヤー	4
ディオ	4
臨場	4
透明度	4
サンゴ礁	4
海へ	4
赤土	4
イオン	4
いわい	4
ホップ	4
雇用促進住宅	4
臨時便	4
江	4
計画経済	4
コンパクトディスク	4
海上空港	4
聴力	4
解読	4
ずん	4
寄せ書き	4
レコード会社	4
紋章	4
呪術	4
門付	4
水路	4
怪談	4
時代劇	4
明け方	4
臣	4
オーディション	4
おじいさん	4
ミサ	4
ドレス	4
運送	4
街灯	4
高所作業車	4
改札	4
すり合わせ	4
フリーダイヤル	4
選日	4
細長	4
不条理	4
世界人口	4
表裏	4
伝記	4
監修	4
オストメイト	4
クラブ活動	4
パーフェクト	4
先発ローテーション	4
ちの町	4
こんなにも	4
情報戦	4
バブル景気	4
卒業証書	4
延焼	4
専決処分	4
管理費	4
ホク	4
洋酒	4
同人	4
集中豪雨	4
実数	4
ゴーストタウン	4
防潮堤	4
債権回収	4
空転	4
省令	4
スピードアップ	4
大動脈	4
コンビナート	4
動議	4
艦	4
大規模開発	4
確信犯	4
量刑	4
エラー	4
ボレー	4
フライパン	4
ターミナルケア	4
の宗教	4
生息地	4
拒絶反応	4
礫	4
真夜中	4
山菜	4
幕府	4
農作物	4
土地区画整理事業	4
運動場	4
サイレン	4
繰り上げ当選	4
型式	4
初等教育	4
大量破壊兵器	4
凶作	4
年上の女	4
問題意識	4
マスク	4
の地下鉄	4
超法規的措置	4
貨物列車	4
睡眠	4
ボランティア元年	4
男役	4
医療費	4
買い占め	4
スーパーマーケット	4
アルファベット	4
カタ	4
からくり	4
和歌	4
判決理由	4
超音波	4
共同購入	4
帝国	4
近代詩	4
篆刻	4
ソファ	4
万能薬	4
同性愛	4
古参	4
簿記

サッカー	2
優勝旗	2
渋み	2
中波	2
宗教番組	2
私人	2
ゴルフ	2
盲学校	2
記	2
気体	2
塗料	2
合成樹脂	2
補欠選挙	2
縄跳び	2
二人三脚	2
造船所	2
結納	2
スキンシップ	2
療養費	2
連呼	2
芸名	2
顔見世	2
水圧	2
誤差	2
鉄鋼業	2
多角化	2
考課	2
成績評価	2
大州	2
寒冷地	2
シャワー	2
戦記	2
潤滑油	2
地球	2
嫌煙	2
パイオニア	2
時代物	2
攻め合い	2
ウリ	2
棋風	2
五歩	2
の格差	2
大通り	2
縮瞳	2
建築確認	2
品種改良	2
貯水槽	2
遊離	2
校門	2
凡人	2
念書	2
保護観察	2
書道展	2
行会	2
無投票当選	2
総合学科	2
専修学校	2
教育行政	2
工業団地	2
茶話会	2
アセスメント	2
健康法	2
占い	2
霊感	2
華美	2
女性の権利	2
銃規制	2
移送	2
監視塔	2
親密さ	2
缶コーヒー	2
鉱石	2
円相場	2
病原体	2
予後	2
本音と建前	2
治験	2
肝臓	2
画集	2
ストライクゾーン	2
番勝負	2
開眼	2
系アニメ	2
神経毒	2
タレント	2
幾何学	2
現代音楽	2
銃の部品	2
ドアミラー	2
簡易裁判所	2
民事調停	2
調停委員	2
コット	2
車庫	2
金融資産	2
公益事業	2
高幹	2
免震	2
ソフトランディング	2
社会的影響	2
ナイトクラブ	2
軍用機	2
モデム	2
パロディ	2
下士官	2
半数致死量	2
玉音放送	2
曖昧	2
神楽歌	2
色鉛筆	2
各駅停車	2
カルチャーショック	2
晴れの日	2
桃源郷	2
テラス	2
カレー	2
ディレクター	2
民社	2
一刀両断	2
教科	2
高等教育	2
情報サービス	2
勉強ができない	2
副読本	2
複数政党制	2
教育委員会	2
雅号	2
塗装	2
玩具	2
両性の合意	2
市内通話	2
静物画	2
チス	2
あいたい	2
ポロポ	2
おたく	2
リタイ	2
まっすぐ	2
捕球	2
スキンヘッド	2
サイクル安打	2
ことわざ	2
ビデオリサーチ	2
国際保護鳥	2
経験則	2
ショールーム	2
里親	2
千年王国	2
灸	2
目録	2
筋交い	2
金貨	2
不純物	2
密輸	2
空気入れ	2
とびきり	2
エステー	2

タ	1
偶数	1
七夕	1
エバンス	1
地球人	1
内覧	1
上奏	1
専攻科	1
テモテ	1
寒剤	1
シンナー	1
差押	1
共同利用	1
固定資産	1
除却	1
一般競争入札	1
海外ドキュメンタリー	1
サマーキャンプ	1
デイケア	1
網戸	1
寝床	1
生鮮食品	1
スポーツセンター	1
中華街	1
兵士	1
同伴者	1
添い寝	1
傷病	1
診断書	1
電子掲示板	1
たこ焼き	1
兵庫県	1
経済学	1
探査機	1
鼻血	1
坊主	1
第三次産業	1
職能給	1
時間外労働	1
度	1
和洋折衷	1
給分	1
算額	1
能格	1
キトー	1
角質	1
基礎化粧品	1
水力	1
クルクル	1
信楽焼	1
訪問販売	1
フォグランプ	1
エンジンオイル	1
粘度指数	1
ランタン	1
オートリ	1
スガイ	1
内間	1
ペナント	1
世襲	1
待ちぼうけ	1
日厳	1
終電	1
北国の	1
ヘリポート	1
生部	1
学生食堂	1
さくらさくら	1
新聞配達	1
ぜんまい仕掛け	1
下駅	1
ラタ	1
ユーラシアン	1
現代詩	1
サイン本	1
はじまり、	1
歴史地震	1
無能	1
呼出	1
ハッカー	1
会理	1
選手宣誓	1
基数	1
燦	1
生活道路	1
加湿器	1
振動板	1
貨物機	1
神経伝達物質	1
領空	1
多目的ダム	1
る沿	1
復刊	1
サケ	1
洪水調節	1
重力式コンクリートダム	1
避難訓練	1
非常食	1
形容詞	1
いまを生きる	1
呪文	1
闇の奥	1
黙示	1
連想ゲーム	1
信販会社	1
写経	1
講式	1
協調性	1
全人教育	1
トピ	1
推薦入学	1
電波塔	1
の原因	1
新市町	1
古名	1
新産業都市	1
余白	1
血液検査	1
尿検査	1
乖離	1
桜前線	1
塩害	1
インチ	1
制度的保障	1
現代病	1
対主	1
日和見主義	1
親衛隊	1
王立	1
夜明け前	1
移民政策	1
革命家	1
プロパガンダ	1
糖化	1
甘味料	1
申告	1
有害物質	1
電動自転車	1
仕事率	1
系人	1
参入障壁	1
感染経路	1
自己決定権	1
ささえ	1
シャト	1
宇宙食	1
遺伝子治療	1
死に至る病	1
角膜	1
説話	1
漆絵	1
和服	1
万華鏡	1
ノックアウト	1
厩舎	1
技あり	1
盗塁

アイディ	1
の壁崩壊	1
悲喜劇	1
妖精	1
口承文学	1
ボーカ	1
フレンドリー	1
レプリカ	1
小目	1
三々	1
ルネサンス	1
本題	1
われら	1
貪	1
家職	1
刀剣	1
禁欲	1
代役	1
刑事法	1
アチーブメントテスト	1
タックスアンサー	1
南地	1
水遊び	1
航空機の検査	1
動物実験	1
義人	1
忙殺	1
耐震工事	1
ちょび	1
勘当	1
おもろ	1
コメディアン	1
編み物	1
自打球	1
の福祉	1
有限会社	1
地方競馬	1
直接販売	1
アッツ島	1
サングラス	1
ゆく河の流れは絶えずして	1
新書	1
いいじゃありませんか	1
基本法	1
岡部三郎	1
院内会派	1
専守防衛	1
コンパクト化	1
山口	1
染色体異常	1
放射線療法	1
声帯	1
ユートピア	1
渡り鳥	1
底	1
債	1
美意識	1
異常気象	1
山地	1
天気図	1
寡占	1
盧	1
民主化宣言	1
抵抗勢力	1
バンパー	1
集会の自由	1
霊視	1
告訴・告発	1
公職	1
断食	1
ヒット商品番付	1
交信	1
公理	1
国民体育大会	1
動物虐待	1
とまん	1
水部	1
議定書	1
生産関係	1
盆栽	1
ログハウス	1
万国旗	1
共同浴場	1
外湯	1
不意打ち	1
茎	1
金策	1
釘	1
・ムーブメント	1
種子島宇宙センター	1
は伝来	1
質問主意書	1
人権教育	1
原子炉格納容器	1
門限	1
父と子	1
間体	1
熱伝導率	1
グラファイト	1
集成材	1
活魚	1
商圏	1
回転率	1
福利厚生	1
職場体験	1
調律	1
防災センター	1
表札	1
私立大	1
国際郵便	1
毒グモ	1
体積	1
高部	1
ごちそうさま	1
サーチライト	1
救命	1
小僧	1
一個人	1
窯跡	1
中公新書	1
るプレゼント	1
国際経済	1
句会	1
自然成立	1
消費者教育	1
乱闘	1
ドクトリン	1
シーソー	1
大判	1
魔法	1
書斎	1
日講	1
国鉄時代	1
ラーマ	1
兄妹	1
コンペティション	1
戦争映画	1
リメー	1
映画製	1
ザッピング	1
宇宙飛行	1
歴史劇	1
ドライアイス	1
作品番号	1
音源	1
時代精神	1
本当の自分	1
セックスシンボル	1
ニコリ	1
幌	1
カメラワーク	1

B-ignoredを考慮したことで、固有名詞を結構取り除けた。

さらに精錬する方法は？

* 形態素解析で分かち書きした境界に一致するか判定

BCCWJでもやってみる。

In [26]:
from entitypedia.evaluation.converter import xml_to_iob2
from seqeval.metrics.sequence_labeling import get_entities
from collections import Counter

bccwj_dir = os.path.join(BASE_DIR, 'bccwj')
X_true, y_true = xml_to_iob2(bccwj_dir, {})
y_pred = [recognizer.iob2(text) for text in X_true]

chars = []
tags = []
for pred, true, doc in zip(y_pred, y_true, X_true):
    for p, t, char in zip(pred, true, doc):
        if t == 'O' and p != 'O':
            chars.append(char)
            tags.append(p)
            
entities = get_entities(tags)

words = []
for _, begin_offset, end_offset in entities:
    end_offset += 1
    word = ''.join(chars[begin_offset: end_offset])
    words.append(word)
    
freq_bccwj = Counter(words)
freq100 = sum(v for _, v in freq_bccwj.most_common(n=100))

print('Num of tokens: {}'.format(len(words)))
print('Num of types  : {}'.format(len(set(words))))
print('{} / {}'.format(freq100, len(words)))

for word, fr in freq_bccwj.most_common():
    print('{}\t{}'.format(word, fr))

Num of tokens: 1023651
Num of types  : 9619
861682 / 1023651
の	69064
に	43308
い	42153
る	36389
た	36082
て	35206
と	34384
で	34002
し	33995
を	33309
は	32220
が	31870
な	30066
っ	20411
か	20380
す	20116
れ	17043
も	16361
ら	15739
ま	15616
う	14309
こ	13985
り	11825
く	10977
だ	10598
あ	9593
き	8675
ん	8220
さ	8095
そ	7871
け	7386
ど	7064
め	6468
え	6069
や	5387
つ	5135
わ	5123
お	4320
せ	3934
よ	3550
ち	3432
よう	3258
ば	3110
み	3038
的	2932
ろ	2716
じ	2640
分	2307
ず	2246
合	1960
方	1899
ね	1871
べ	1704
事	1689
場	1676
要	1661
へ	1658
対	1641
び	1633
ほ	1553
とも	1481
げ	1466
定	1387
より	1364
私	1343
体	1332
ょ	1316
日	1249
取	1245
手	1224
間	1186
何	1178
本	1128
化	1123
内	1122
む	1110
年	1106
気	1094
ゃ	1048
受	1013
目	1012
業	969
新	952
国	931
ご	915
なかった	909
数	838
向	833
所	825
ぶ	799
備	775
金	761
連	759
ぐ	752
施	741
ですか	735
地	717
強	689
無	673
におい	660
心	650
調	633
情報	627
ぎ	626
問題	613
づ	597
理	573
保	573
食	572
企業	570
はい	568
のあ	566
まれ	561
有	545
りの	544
ひ	538
いない	530
活動	528
明	521
外	515
水	500
店	492
引	490
結	485
地域	476
指	472
可能	470
親	453
時間	446
ゆ	444
ふ	437
止	435
運	432
対策	421
常	413

興奮	19
乾	19
おまけ	19
才	19
洗濯	19
不動産	19
寄付	19
ぅ	19
強度	19
休日	19
留学	19
物事	19
りお	19
給食	19
ママ	19
氏名	19
不幸	19
出張	19
暫	19
中核	19
敷地	19
立地	19
ら行	19
高等専門学校	19
首	19
生態系	19
まちづくり	19
間伐材	19
リサイクル	19
運搬	19
資質	19
公平	19
収穫	19
重さ	19
貧困	19
水源	19
定員	19
ジュール	19
出力	19
漁	19
見解	19
庶民	19
純粋	19
恋人	19
先祖	19
占領	19
航海	19
城下町	19
立候補	19
クリー	18
作者	18
諦	18
オンライン	18
掲示板	18
ネー	18
経由	18
職業訓練	18
自覚	18
遠慮	18
強烈	18
気象	18
待った	18
新鮮	18
用語	18
上下	18
傘	18
ボロ	18
受け入れて	18
あなたと	18
抗議	18
まっすぐ	18
土産	18
スクリーン	18
発達	18
液体	18
気温	18
あっせん	18
管轄	18
公益	18
動員	18
法令	18
トロール	18
課程	18
長所	18
奉仕	18
患	18
記者会見	18
国土	18
航空	18
旅客	18
データベース	18
パイプ	18
公判	18
本質	18
ファッション	18
悲劇	18
合掌	18
行方不明	18
ビジネスモデル	18
花見	18
女たち	18
不祥事	18
競売	18
前売り	18
凡	17
看板	17
ピアノ	17
らいな	17
コント	17
原作	17
セキュリティ	17
レシピ	17
斜面	17
実習	17
恨	17
定義	17
人権	17
操縦	17
缶	17
スム	17
テレビ番組	17
預金	17
家具	17
保障	17
夕食	17
奇	17
測定	17
いじめ	17
最中	17
国籍	17
言語	17
文明	17
ライバル	17
イドラ	17
ペットボトル	17
段ボール	17
抽出	17
研究会	17
譲渡	17
ヘリコプター	17
景観	17
破壊	17
見学	17
浸透	17
施工	17
解体	17
諮問	17
べる。	17
水辺	17
絵画	17
夏	17
太鼓	17
名所	17
境内	17
ヒト

慣習	4
岸壁	4
心中	4
世話人	4
ショッピングセンター	4
社会保険	4
クープ	4
まなざし	4
嫉妬	4
咀嚼	4
脳	4
輝いた	4
はさみ	4
天領	4
報道番組	4
小京都	4
清流	4
心底	4
マウン	4
徴用	4
柵	4
たち	4
ジャズ	4
起点	4
予定利率	4
確信犯	4
播種	4
宇宙ステーション	4
クレーター	4
探検	4
宇宙空間	4
夕刊	4
目論見書	4
債務超過	4
ラフト	4
アルファベット	4
ポケ	4
卸売	4
ベンチマーク	4
注射	4
右翼	4
和風	4
寓話	4
回診	4
中	4
綱領	4
スモー	4
パケット	4
社内報	4
小冊子	4
イラストレーション	4
ソープランド	4
古都	4
薫	4
溶岩台地	4
オープン価格	4
庭園	4
虚空	4
名義	4
アトリエ	4
スコープ	4
ミニバン	4
世界一周	4
学期	4
除名	4
賓	4
役職	4
反粒子	4
略奪	4
捕虜	4
退学	4
憎悪	4
ラエ	4
同盟国	4
通関	4
ミイラ	4
指名手配	4
正論	4
金融資産	4
準々決勝	4
タミ	4
食肉	4
遺作	4
姉妹都市	4
質疑	4
労働組合	4
民族紛争	4
財政再建	4
無形文化財	4
会派	4
人工授精	4
慣用句	4
業務改善命令	4
納屋	4
借景	4
監査法人	4
走者	4
初演	4
心部	4
タイガ	4
知行	4
国際電話	4
出来高	4
パネルディスカッション	4
世代交代	4
若返り	4
上告	4
海の家	4
持ち時間	4
凶器	4
配列	4
故意	4
過激派	4
信託	4
市場価格	4
歴史認識	4
海運	4
建築確認	4
節税	4
戦中	3
戦後	3
電報	3
サビ	3
タラ	3
ヶ	3
コルク	3
誰も知らない	3
詞	3
ボーカル	3
この	3
週刊誌	3
糞	3
ラジオ番組	3
新刊	3
マスク	3
萌え	3
アップロード	3
レジストリ	3
地域密着	3
未解決	3
チャット	3
再起動	3
ログイン	3
チューナー	3
フォント	3
引数	3
イコン	3
地名	3
四苦八苦	3
レーベ	3
ダクト	3
プログラミン	3
完結	3
扇風機	3
灯油	3
解像度	3
音域	3
感度	3
お手上げ	3
持病	3
お葬式	3
自己満足	3
病歴	3
情報操作	3
競争原理

みっけ	1
からくり時計	1
模糊	1
語尾	1
次の内閣	1
悲惨な戦争	1
乃	1
強行採決	1
おから	1
ランドセル	1
住民自治	1
受刑者	1
雨漏り	1
耕地整理	1
無所属	1
投票行動	1
党首	1
刺激物	1
の頃	1
睡眠薬	1
パワーストーン	1
入出	1
スクワット	1
カロリー	1
モノス	1
お召	1
必須脂肪酸	1
イケア	1
たるみ	1
脂肪	1
シャーベット	1
フェロモン	1
特別支援教育	1
メンタルヘルス	1
鉄格子	1
基本ブロック	1
木棺	1
石棺	1
何処へ行く	1
りょう	1
ペアレンツ	1
大奥	1
修羅場	1
魔女	1
おうちでごはん	1
連絡網	1
やる気満々	1
鞄	1
ちぃ	1
科属	1
有毒植物	1
重要美術品	1
遠くの空	1
キャッホー	1
急ブレーキ	1
稲刈り	1
幼鳥	1
思いやり予算	1
世界遺産	1
スイートルーム	1
リンス	1
バイオ燃料	1
三脚	1
急流	1
幾何学模様	1
腕章	1
山伏	1
演武	1
お遊戯	1
ペロリ	1
肩車	1
木登り	1
沽券	1
日の後	1
ホワイトバランス	1
接写	1
高架橋	1
魔界	1
還付金詐欺	1
男湯	1
歩いてる	1
コスモス	1
コスモス街道	1
の名前	1
パカ	1
魏略	1
難升米	1
詔	1
不思議な世界	1
ウォーキング	1
ぽーぽー	1
コーチング	1
アンテナショップ	1
リベンジ	1
和紙	1
インカー	1
連作	1
スメラ	1
スロットル	1
チョーク	1
パンパン	1
釣り堀	1
針外し	1
クーラーボックス	1
誹謗中傷	1
しりとり	1
まった	1
あるじゃん	1
詩吟	1
ウトウ	1
物真似	1
裏声	1
凛々	1
牽制球	1
き裂	1
春霞	1
ひとりぼっち	1
芸能人	1
旅の歌	1
志向性	1
ネタバレ	1
変態	1
豚肉	1
兄弟姉妹	1
息災	1
英才	1
ブルブル	1
晶	1
サービスセンター	1
るいか	1
子供会	1
プラネタリウム	1
夕焼け	1
また戻ってくる	1
オーバーレブ	1
バラバラ殺人	1
運命の恋人	1
通り魔	1
ネットサーフィン	1
破局	1
チゲ	1
飛び道具	1
チセ	1
ダンジョン	1
強くてニューゲーム	1
ジャギー	1
スクリーンショット	1
グラフィック	1
イン

毎日新聞とBCCWJの共通部分を洗い出す。

In [27]:
len(set(freq.keys()) & set(freq_bccwj.keys()))

6671

毎日だけにある単語

In [28]:
print(len(set(freq.keys()) - set(freq_bccwj.keys())))
for word in set(freq.keys()) - set(freq_bccwj.keys()):
    print(word)

4435
星雲
工
天然繊維
走
はの核実験
あきな
マクラ
光エネルギー
確証
マッチレース
重戦車
捨て子
健康法
自動車道
民芸品
化学式
株式ミニ投資
櫂
征伐
輸送人員
コレクトコール
近代詩
スエ
脳死
常民
抑止
帯封
行水
調律
水筒
モスラ
の原因
暗算
縮尺
痴漢
現代病
マスゲーム
最低賃金
切腹
人称
植物園
任官
知覚
塩漬け
会
鋼板
航空機の検査
シリンダー
救命いかだ
上水道
公募展
沐浴
図鑑
分割統治
事情判決
羽毛
ポータブルトイレ
タイプライター
正統性
磁器
施政方針演説
第二芸術
政庁
レプリカ
各駅停車
原人
豪農
スキンシップ
面食い
名望家
条約改正
最終便
地域医療
氏寺
爆竹
リスク回避
近親相姦
特打
自助具
脂肪組織
帯番組
桜前線
槍術
女の歌
プリズム
暴投
金縛り
三世
世界は変わる
呪文
合性論
フレグラ
ユニセックス
超能力
寄生虫
中道右派
鬼畜
俳号
ミニコンポ
この声
替え歌
元老
海上封鎖
偶像崇拝
大州
すいとん
むぐら
任意同行
景気循環
高架鉄道
戦史叢書
スガイ
直接投資
裁決
照度
埋蔵文化財
アクラ
フナ
南画
同和教育
ハイボール
水産業
手の目
火の粉
カースト
スケッチブック
元素合成
台湾料理
内供
真空地帯
伝染病
民族共同体
活字離れ
向け
電気泳動
ゲド
テレビデオ
や会
座り込み
決まり文句
衛生陶器
私年号
当たり屋
競馬場
火災報知機
散文
銃規制
北勢
漆絵
お色気番組
神戸市
ビッチ
頭突き
正書法
社
政官
乗換駅
公邸
ナーシ
国家補償
皮革
屋根裏
検出限界
みんな元気
外務委員会
チェンバロ
仕事率
開放経済
衰弱死
マントラ
めくり
ホットライン
殺し屋
タンパ
雑損控除
バーゲニング・パワー
美人画
粉飾
ノミネート
原
素描
超新星
シンボルマーク
無煙火薬
ホル
紙飛行機
直方体
仏師
コート
ドアミラー
やつし
住民基本台帳
鳩
調剤
親衛隊
上場廃止
る・
事実認定
性癖
アラカルト
トンコリ
爆発物
奈良
レーザーディスク
実相
参入障壁
変電設備
チャント
緑肥
線香花火
防犯灯
圏
ガラス繊維
小旗
医科大学
中央高速道路
在家
宮殿
人口密度
俵
団体総合
支援戦闘機
べの住宅
阿頼耶識
ベベ
闘牛
ボックス・セット
麻雀
火葬

未来のために
宇宙食
リスナー
為政
雑踏警備
微罪逮捕
セミナー
耐力壁
放送衛星
分子構造
デンカ
元幹
労働金庫
武装闘争
艦砲射撃
名画座
逆差別
グリーンコンシューマー
証
集音器
オートリ
漫才
問題集
政治哲学
演題
こいのぼり
王族
原子核
やませ
アルク
水冷
暦法
幕下
救援車
日こう
キャラバン
共同企業体
合従連衡
熱伝導率
一
教務
幽霊屋敷
護法
黒石
アニマ
機関投資家
大天狗
財政政策
亜種
竪琴
茶髪
がぶり
完成予想図
視程
前方後円墳
進攻
短資会社
油圧
毎回得点
ゲリラ闘争
ロマンチック・ラブ
ディスポーザー
標準軌
核戦略
今を生きる
番狂わせ
先発ローテーション
富士見書房
逮
盤寿
針路
帰省ラッシュ
ヨシ
ピロティ
インティ
自己認識
打ち
どうにかなるさ
グスク
集団検診
新星
ベストメンバー
性差別
時の旅人
村山
官報
オール与党
雅楽
い抑止
口承文学
感染経路
指人形
厚生年金基金
コン部
おとり捜査
番兵
供託
約定
ノンプライム
公共職業安定所
絹織物
皇族
運動公園
ジェト
バイパス道路
災害対策特別委員会
夢のひとつ
ツンドラ
マウスピース
金物
大理石
スコール
寝袋
映画人
ミニゲーム
横綱
河川敷
教育権
栄養学
アレルギー
基礎化粧品
動く歩道
キロ
解釈学
ボイラー室
穀
気圧
抑留
続く世界
土下座
エンパワーメント
表決
デモテープ
遠隔操作
暖冬
推論
気象観測
師団
となりの女
はしご酒
価格戦略
ポロ
プレス機
角膜
火の車
共同溝
竹細工
食糧費
シデ
フィードバック
限
ヒット商品番付
南地
の基本定理
戦病死
需要と供給
一筆書き
常温核融合
街頭募金
コンペティション
微風
モノカルチャー
能格
運営委員会
写真フィルム
求婚
スターボー
三国
動物虐待
肝臓
多数決
反感情
改姓
解脱
レジスタンス運動
強制加入
道楽
ケミ
飛行禁止空域
打ち切り
猪突猛進
日講
怪力
投資銀行
内部障害
看護学
殺処分
自治権
修繕
なとり
建築確認申請
有害物質
義歯
有期刑
証文
ブライダ
世情
分注
系アニメ
除草剤
挙国一致内閣
洋楽
開発独裁
ダンスパーティー
守秘義務
化学合成
シーマン
別件逮捕
血糖値
夜逃げ
食客
性犯罪
社団
スタッドレスタイヤ
裃
戦争映画
霊歌

BCCWJだけにある単語

In [29]:
print(len(set(freq_bccwj.keys()) - set(freq.keys())))
for word in set(freq_bccwj.keys()) - set(freq.keys()):
    print(word)

2948
不完全燃焼
小型船舶
保護房
単項式
サラリーマン
盃
居城
パテントプール
エクサ
吸虫
雑巾
天下一品
有価証券報告書
火山弾
無線電話
演武
来島
オリジナルカード
なんだかんだ
町内会
クレンジング
アフィ
紐
静電気
沽券
あなた。
幼児語
ゴマ
入梅
罠
しんじ
ローカル線
手事
レタリング
不安定現象
精密機械
ソープランド
巨大津波
現代語
瀬戸際政策
時の娘
毛織物
夏休み
おおきな
転送速度
多角形
添付ファイル
什器
ザマ
副業
友達親子
咸宜
まにまに
治山
インターネット電話
プログラミン
副反応
パチスロ
茶箱
勘定科目
民法
ジャッキ
位牌
食品表示
気管
灌漑
ドラムブレーキ
補聴器
避難指示
鍛造
歴史小説
右傾
ベリー
貨客船
旅籠
ジョイスティック
スキー場
天秤棒
クロック
天上天下
英才
再現ドラマ
霊界
ラブホテル
ちぃ
地すべり
基礎代謝
三顧の礼
新地
戦後
カービング
戦区
結合組織
オミ
ヶ
啓示
中華鍋
旅の歌
化身
侍
と動物
小春日和
釣り堀
タイガ
ゼミナール
異境
材木問屋
格子
真理値表
企業倫理
賄賂
確保
オープンソース
高速自動車国道
団子
届出制
雨乞い
のロック
コミュニティ放送
の山
モケット
認定農業者
太古
人工林
ファイル共有ソフト
幟
暗殺
ローカル局
裏番組
価指数
靱
魚粉
手押し車
ツゲ
中間法人
ウイスキー
貨幣
春霞
愛の予感
排他的経済水域
リキュール
ため池
労働者派遣事業
モジュール
承継銀行
余談
来し方行く末
鴨居
はとこ
社員食堂
お先に失礼
動物霊
会計基準
キラキラ☆
純喫茶
パタ
無職
壺
外国法人
うらぎり
海図
アットホーム
ハハハ
ロロ
フリーフォント
手摺
水着
もういちど
神秘体験
前衛党
複合商業施設
校庭
台地
発達段階
農楽
計算ちがい
ドアノブ
金切り鋏
共有地
エクステリア
もらい泣き
オリテ
家船
回路設計
デバッグ
ハードディスクドライブ
棚田
障害者差別
かりあげ
不沈空母
公道
性的快感
インディアカ
たたき上げ
偶像
板紙
綿棒
お祭り騒ぎ
スープ
フォルム
鉤
ノゾキ
社会的認知
捜査官
割り箸
新字体
諸色
ダウンロード
審査請求
朝陽
消防学校
妾
白砂青松
加入電話
健康酒
遺族年金
真空状態
髪型
学
大股

気になる点

* 曖昧さ回避ページとどのくらいかぶるのか？
  * 曖昧さ回避ページを取り除いてから教師データ作るとラベル付する数が減らせるのでよい。